In [ ]:
"""
We've got a mapreduce job that parses through XML files to find every 
unique combination of tags. We've also got some scripts to grab the 
results from S3 and write it to s3 or a csv. 

We need to look through these paths to find the queries that we
care about. For this first bit of scratch code, we'll look for paths
that include `name` but exclude `business`.

The output of this process will be a .csv file 
"""
import pandas as pd

In [ ]:
df = pd.read_csv('schema_map.csv', sep='|', names=['form_version_id', 'path'])
df['element_name'] = df.path.str.split('.').apply(lambda x: x[-1])

df.head()

In [ ]:
df['has_name'] = df.element_name.str.lower().str.contains('name')
df['has_business'] = df.element_name.str.lower().str.contains('business')

names = df[(df.has_name) & (~df.has_business)]
# fvi is form_version_id
frequency_of_fvi = names.groupby('path')['element_name'].count()

In [ ]:
names.set_index('path', inplace=True, drop=False)
names = names.assign(frequency=frequency_of_fvi)
names.sort_values(by='frequency', ascending=False)

In [ ]:
names.to_csv('paths.csv', index=False)